In [110]:
%run APYME_bd.ipynb
%run APYME_Modelo.ipynb

path = '/Users/mb66483/Downloads/Crédito PyME/'


In [111]:
df_universo, df_sunat, df_sunart, df_rcc, df_ranking, df_digital, df_transac = read_bd(path)


db_ranking = clean_db_ranking(df_ranking)
db_ranking.sample(5)

df_sunart_1 = transf_sunart()
df_rcc_5 = transf_rcc()
df_digital = transf_digital(df_digital)



In [113]:
col_ranking=['taxpayer_id','seniority_company_years_number',
       'company_size_desc', 'foreign_company_mark_type', 'department_name',
       'province_name', 'district_name', 'private_public_mark_desc',
       'market_share_per', 'ranking_last_number',
       'min_usd_billing_last_amount', 'max_usd_billing_last_amount',
       'min_pen_billing_last_amount', 'max_pen_billing_last_amount',
       'exporter_mark_type_flag', 'importer_mark_type_flag',
       'pbi_sector_last_per']    

col_digital = ['customer_id','u_digital_affiliation_type','u_online_banking_affiliation_type','u_sms_affiliation_type',
               'dias_digital','dias_online','dias_sms']

db_ranking2 = db_ranking.fillna(0)
db_ranking2 = db_ranking2.drop_duplicates()

df_join = df_universo.merge(df_sunat, how='left', on='taxpayer_id')\
            .merge(df_sunart_1, how='left', left_on='personal_id_x', right_on='personal_id')\
            .merge(df_rcc_5, how='inner', on='sbs_customer_id')\
            .merge(db_ranking2[col_ranking], how='left', on='taxpayer_id')\
            .merge(df_digital[col_digital], how='left', on='customer_id')


In [54]:
df0 = pd.DataFrame(db_ranking['taxpayer_id'].value_counts()).reset_index()

In [114]:
dbTransacPyme['Trasp_BBVA'] = np.where( (dbTransacPyme['COD_PERSONOR'].notnull()) & (dbTransacPyme['COD_PERSONDE'].notnull()), 
                                     1, # Si persona origen y destino no son NaN -> 1
                                     0  # Caso contrario -> 0
                                   )
& (df_transac.COD_PERSONDE.isnull())

,personal_type_x,personal_id_x,customer_id,sbs_customer_id,taxpayer_id,segmento,gender_type,birth_date,marital_status_type,admission_date,...,max_pen_billing_last_amount,exporter_mark_type_flag,importer_mark_type_flag,pbi_sector_last_per,u_digital_affiliation_type,u_online_banking_affiliation_type,u_sms_affiliation_type,dias_digital,dias_online,dias_sms
0,L,90048822,98314621,6200576519,81285667393,PNN,M,1961.0,S,2019-01-10,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,126.0,126.0,414.0
1,L,71275253,64058884,4574824240,08549444909,PNN,M,1960.0,C,1996-08-01,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,632.0,632.0,708.0
2,L,39025845,44955658,8268872505,11854064724,PNN,M,1962.0,C,1998-11-13,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1201.0,1201.0,1201.0
3,L,29398539,01707393,7974280548,38837860896,PNN,F,1962.0,C,1998-05-22,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,224.0,224.0,224.0
4,L,83084480,67646864,5908385493,20489054918,PNN,F,1962.0,C,2018-06-15,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,539.0,539.0,539.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38142,L,26019586,91420350,7047881822,65192962924,PNN,F,1989.0,S,2016-02-05,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,525.0,525.0,674.0
38143,L,76048604,72189034,7519133747,40489744700,PNN,M,1990.0,S,2019-08-31,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,181.0,181.0,181.0
38144,L,76048604,72189034,7519133747,40489744700,PNN,M,1990.0,S,2019-08-31,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,181.0,181.0,181.0
38145,L,61984330,80228448,4227844091,39612419378,PNN,M,1994.0,S,2016-09-09,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,307.0,307.0,320.0


In [163]:
df_transac['FEC_OPER_d'] = pd.to_datetime(df_transac['FEC_OPER'], format='%d/%m/%Y')

df_transac.loc[df_transac.COD_PERSONOR==df_transac.COD_PERSONDE,'tipo']='traspaso_ctas'
df_transac.loc[ (df_transac.COD_PERSONOR.notnull()) & (df_transac.COD_PERSONDE.isnull()) ,'tipo']='trasf_env'
df_transac.loc[ (df_transac.COD_PERSONOR.isnull()) & (df_transac.COD_PERSONDE.notnull()) ,'tipo']='trasf_rec'
df_transac.loc[ (df_transac.COD_PERSONOR!=df_transac.COD_PERSONDE) ,'tipo']='traspaso_ctas_otro'



In [175]:
df_transac['tipo2'] = np.where(df_transac.COD_PERSONOR==df_transac.COD_PERSONDE,'traspaso_ctas',\
                        np.where((df_transac.COD_PERSONOR.notnull()) & (df_transac.COD_PERSONDE.isnull()),'trasf_env',\
                        np.where((df_transac.COD_PERSONOR.isnull()) & (df_transac.COD_PERSONDE.notnull()),'trasf_rec',\
                        np.where((df_transac.COD_PERSONOR!=df_transac.COD_PERSONDE),'trasf_rec',0))))
                     

In [176]:
df_transac.groupby('tipo2').count()

,COD_PERSONOR,COD_PERSONDE,IDW_CANAL,COD_TRANSAC,FEC_OPER,HMS_OPER,IMP_IMPOPER,FEC_OPER_d,tipo
tipo2,,,,,,,,,
trasf_env,2920287,0,2920287,2920287,2920287,2920287,2920287,2920287,2920287
trasf_rec,9861435,9862102,9862102,9862102,9862102,9862102,9862102,9862102,9862102
traspaso_ctas,240808,240808,240808,240808,240808,240808,240808,240808,240808


In [148]:
df_transac[df_transac.COD_PERSONDE.isnull()]

,COD_PERSONOR,COD_PERSONDE,IDW_CANAL,COD_TRANSAC,FEC_OPER,HMS_OPER,IMP_IMPOPER,FEC_OPER_d,tipo
0,29586275,NaN,14,FN77,01/09/2019,163738,29.9,2019-09-01,traspaso_ctas
1,58487151,NaN,14,FN83,01/09/2019,163935,69.9,2019-09-01,traspaso_ctas
2,49826224,NaN,14,FN51,01/09/2019,163147,60.0,2019-09-01,traspaso_ctas
3,34555045,NaN,14,FN9A,01/09/2019,163617,3.0,2019-09-01,traspaso_ctas
4,42854084,NaN,14,FN77,01/09/2019,163907,390.0,2019-09-01,traspaso_ctas
...,...,...,...,...,...,...,...,...,...
14963604,44825759,NaN,18,KW70,28/06/2020,185418,193.0,2020-06-28,traspaso_ctas
14963605,40921837,NaN,18,KW87,28/06/2020,122722,484.4,2020-06-28,traspaso_ctas
14963606,40921837,NaN,18,KW87,28/06/2020,123251,21.2,2020-06-28,traspaso_ctas
14963607,26439723,NaN,18,KW98,28/06/2020,180325,79.9,2020-06-28,traspaso_ctas


In [137]:
df_transac[df_transac.COD_PERSONDE.isnull() & df_transac.COD_PERSONOR.isnull()]

,COD_PERSONOR,COD_PERSONDE,IDW_CANAL,COD_TRANSAC,FEC_OPER,HMS_OPER,IMP_IMPOPER,FEC_OPER_d


In [ ]:
df_transac.groupby('COD_PERSONOR').agg()

In [133]:
df_transac[df_transac.COD_PERSONDE==df_transac.COD_PERSONOR]

,COD_PERSONOR,COD_PERSONDE,IDW_CANAL,COD_TRANSAC,FEC_OPER,HMS_OPER,IMP_IMPOPER,FEC_OPER_d
3153,70339158,70339158,18,KW50,01/09/2019,213707,2200.0,2019-09-01
3154,70339158,70339158,18,KW50,01/09/2019,213900,2200.0,2019-09-01
3155,70339158,70339158,18,KW50,01/09/2019,214216,200.0,2019-09-01
3187,39955461,39955461,18,KW50,01/09/2019,133838,1400.0,2019-09-01
3188,39955461,39955461,18,KW50,01/09/2019,172453,50.0,2019-09-01
...,...,...,...,...,...,...,...,...
14960630,45790902,45790902,18,KX50,28/06/2020,140342,3000.0,2020-06-28
14960631,45790902,45790902,18,KX50,28/06/2020,140436,3000.0,2020-06-28
14960632,45790902,45790902,18,KX50,28/06/2020,140523,3000.0,2020-06-28
14960633,45790902,45790902,18,KX50,28/06/2020,140612,3000.0,2020-06-28


In [126]:
df_transac.groupby('FEC_OPER_d').max()

AssertionError: 

In [131]:
df_transac.groupby('FEC_OPER').count().to_excel('/Users/mb66483/Documents/outx.xlsx')

In [127]:
df_transac.dtypes

COD_PERSONOR            object
COD_PERSONDE            object
IDW_CANAL                int64
COD_TRANSAC             object
FEC_OPER                object
HMS_OPER                 int64
IMP_IMPOPER            float64
FEC_OPER_d      datetime64[ns]
dtype: object

In [70]:
db_ranking

,taxpayer_id,company_economic_activity_id,ciiu_spanish_desc,spanish_sector_company_name,employees_number,branches_number,activity_start_date,seniority_company_years_number,company_size_desc,foreign_company_mark_type,...,private_public_mark_desc,market_share_per,ranking_last_number,min_usd_billing_last_amount,max_usd_billing_last_amount,min_pen_billing_last_amount,max_pen_billing_last_amount,exporter_mark_type_flag,importer_mark_type_flag,pbi_sector_last_per
0,47019243321,113,"Cultivo de frutas, nueces, plantas cuyas hojas...",AGROINDUSTRIA Y GANADERIA,22.0,1.0,2013-10-21,5.0,Mediana empresa / Medium-size Company,0,...,PRIVADA,0.221119,9228.0,2666667.0,3034947.578,9500000.0,9899999.0,EXPORTADORA,IMPORTADORA,2.4
1,45879845584,5190,Venta al por mayor de otros productos,COMERCIALIZADORAS,5.0,1.0,2012-11-01,6.0,Mediana empresa / Medium-size Company,0,...,PRIVADA,0.013783,9652.0,0.0,2912323.421,9000000.0,9499999.0,0,0,0.0
2,62707602419,4520,ConstrucciÃ³n de edificios completos o de part...,CONSTRUCTORAS,7.0,1.0,2001-08-20,17.0,Mediana empresa / Medium-size Company,0,...,PRIVADA,0.019521,0.0,0.0,0.000,0.0,0.0,0,0,0.0
4,79944413565,6023,Transporte de carga por carretera,TRANSPORTE,13.0,1.0,2011-01-09,7.0,PequeÃ±a empresa / Small-size Company,0,...,PRIVADA,0.045532,0.0,0.0,0.000,0.0,0.0,0,0,0.0
5,10200298689,1120,Actividades de servicios relacionadas con la e...,HIDROCARBUROS,0.0,2.0,2004-04-01,14.0,PequeÃ±a empresa / Small-size Company,0,...,PRIVADA,0.142790,9884.0,0.0,2856246.000,8700000.0,9099999.0,0,IMPORTADORA,6.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5947,56395637018,4520,ConstrucciÃ³n de edificios completos o de part...,CONSTRUCTORAS,0.0,1.0,2008-08-22,10.0,Mediana empresa / Medium-size Company,0,...,PRIVADA,0.019521,0.0,0.0,0.000,0.0,0.0,0,0,0.0
5948,23473694594,1810,"FabricaciÃ³n de prendas de vestir, excepto pre...",TEXTILES,4.0,1.0,2006-03-01,12.0,PequeÃ±a empresa / Small-size Company,0,...,PRIVADA,0.175037,0.0,0.0,0.000,0.0,0.0,0,IMPORTADORA,2.0
5949,27688131871,2429,FabricaciÃ³n de otros productos quÃ­micos n.c.p.,QUIMICAS,20.0,1.0,1977-01-02,41.0,Mediana empresa / Medium-size Company,0,...,PRIVADA,0.333218,9058.0,3229630.0,3034947.578,9500000.0,9899999.0,EXPORTADORA,IMPORTADORA,1.0
5950,20064036061,4520,ConstrucciÃ³n de edificios completos o de part...,CONSTRUCTORAS,1.0,1.0,2015-07-17,3.0,PequeÃ±a empresa / Small-size Company,0,...,PRIVADA,0.040126,5379.0,5333333.0,5629629.000,18000000.0,18999999.0,0,0,0.0


In [38]:
db_ranking[db_ranking.taxpayer_id=='48960833818'].to_excel('/Users/mb66483/Documents/out.xlsx')

In [42]:
df_join

,personal_type_x,personal_id_x,customer_id,sbs_customer_id,taxpayer_id,segmento,gender_type,birth_date,marital_status_type,admission_date,...,private_public_mark_desc,market_share_per,ranking_last_number,min_usd_billing_last_amount,max_usd_billing_last_amount,min_pen_billing_last_amount,max_pen_billing_last_amount,exporter_mark_type_flag,importer_mark_type_flag,pbi_sector_last_per
0,L,90048822,98314621,6200576519,81285667393,PNN,M,1961.0,S,2019-01-10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,L,71275253,64058884,4574824240,08549444909,PNN,M,1960.0,C,1996-08-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,L,39025845,44955658,8268872505,11854064724,PNN,M,1962.0,C,1998-11-13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,L,29398539,01707393,7974280548,38837860896,PNN,F,1962.0,C,1998-05-22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,L,83084480,67646864,5908385493,20489054918,PNN,F,1962.0,C,2018-06-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38142,L,26019586,91420350,7047881822,65192962924,PNN,F,1989.0,S,2016-02-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38143,L,76048604,72189034,7519133747,40489744700,PNN,M,1990.0,S,2019-08-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38144,L,76048604,72189034,7519133747,40489744700,PNN,M,1990.0,S,2019-08-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38145,L,61984330,80228448,4227844091,39612419378,PNN,M,1994.0,S,2016-09-09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df_join = join_tables()
df_join = df_join[(df_join.veh_ct>0) & df_join.income_monthly_amount]

In [4]:
df = df_join
df_features = ['income_monthly_amount','birth_date','veh_ct','veh_year']
vo = 'vo_balance_amount'

X_train, X_test, y_train, y_test = partition(df,df_features,vo)
y_train =pd.DataFrame(y_train,columns=['vo_balance_amount'])

In [241]:
random_state=1
activation='relu'
y_predict, regr = model_nn(X_train2, X_test2, y_train2.ravel(), random_state, activation)

In [13]:
#dfres2T, dfres3T = pd.DataFrame(), pd.DataFrame()
dfres2A, dfres3A = pd.DataFrame(), pd.DataFrame()
error=100000000
transformacion = ['minmax','standard','robust']
for transf in transformacion:
    print('//TRANSFORMACION: '+transf)
#print('BANCO: '+banc+'//LAGS: '+ str(lags)+'//VARIABLE: '+column+'//TRANSFORMACION: '+transf)
    X_train2, X_test2, y_train2, mm_scaler, mm_scaler2 = ftransformacion(X_train, X_test, y_train, transf)
    X_train2 =X_train
    X_test2= X_test
    y_train2 = y_train
    
    activation = ['identity','relu']
    for act in activation:
        for i in range(0,10):

            y_predict, regr = model_nn(X_train2, X_test2, y_train2, i,act)
            #y_predict = transformacion_inversa(y_predict2, mm_scaler2)
            dfcom2, dfcom3 =  resultados(df)

            error_i = dfcom3
            #print(error_i[0])
            if error_i<error:

                dfres2A = dfcom2
                dfres3A = dfcom3   

                print("    activation: "+act+" i: "+str(i)+' error: '+ str(error) + " error_i: "+str(error_i))
                error = dfcom3   
#dfres2T = dfres2T.append(dfres2A,ignore_index=True)
#dfres3T = dfres3T.append(dfres3A,ignore_index=True)        


//TRANSFORMACION: minmax


/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


    activation: identity i: 0 error: 100000000 error_i: 1775247.0641145837


/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


    activation: identity i: 1 error: 1775247.0641145837 error_i: 1773974.8094461453


...

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y 

//TRANSFORMACION: robust


/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exampl

    activation: identity i: 2 error: 1103080.9889068976 error_i: 4825.828194322279


/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklea

In [15]:
df_universo

,personal_type,personal_id,customer_id,sbs_customer_id,taxpayer_id,segmento,gender_type,birth_date,marital_status_type,admission_date,income_monthly_amount,cutoff_date
0,L,81114488,61545664,7.184693e+06,37374480882,PNN,M,1954.0,C,2017-10-13,1879.00,2020-02-29
1,L,91105896,89063429,4.024562e+09,50856707388,PNN,M,1970.0,C,2014-07-02,1038.63,2020-02-29
2,L,55014034,83739356,5.775967e+09,79362980482,PNN,M,1972.0,C,1997-09-13,1299.13,2020-02-29
3,L,16587579,92772176,8.446842e+09,11433585070,PNN,M,1961.0,S,2011-01-14,1796.59,2020-02-29
4,L,20167595,48723914,1.983932e+09,96458143025,PNN,F,1965.0,S,1999-07-22,1124.50,2020-02-29
...,...,...,...,...,...,...,...,...,...,...,...,...
268799,L,84724544,51875730,4.274432e+09,3167199217,PNN,M,1991.0,S,2017-03-08,1833.92,2020-02-29
268800,L,61984330,80228448,4.227844e+09,39612419378,PNN,M,1994.0,S,2016-09-09,1098.51,2020-02-29
268801,L,28417002,26231191,2.558686e+09,19916720951,PNN,M,1994.0,S,2016-07-14,1931.56,2020-02-29
268802,L,47284690,24383761,3.392091e+09,74970123245,PNN,M,1966.0,C,2009-10-30,1544.58,2020-02-29


In [166]:
pd.DataFrame(y_train2, columns=["valor_pred"])

,valor_pred
0,-0.209923
1,-0.129151
2,1.981855
3,0.211106
4,-29.934441
...,...
1523,1.101668
1524,-0.136979
1525,-0.278959
1526,-0.596584


In [124]:
df=df_join
dfcom2, dfcom3 = resultados(df)

In [125]:
dfcom2

,index,personal_id_x,customer_id,sbs_customer_id,taxpayer_id,segmento,gender_type,vo_balance_amount,valor_pred,error_est,p_error_est
0,2,39025845,44955658,8.268873e+09,11854064724,PNN,M,3339249.09,800964.894291,2.538284e+06,0.760136
1,6,9350890,36781895,1.686121e+08,40040624596,PNN,M,1176278.00,254280.911167,9.219971e+05,0.783826
2,7,49400873,18301674,3.731068e+09,72277039581,PNN,F,4049624.46,280985.802973,3.768639e+06,0.930614
3,11,54413292,3019627,2.074224e+09,73408383097,PNN,M,-211.91,323503.788191,-3.237157e+05,1527.609354
4,14,19121285,15208732,3.901088e+09,85993190072,PNN,F,273657.61,285333.370029,-1.167576e+04,0.042666
...,...,...,...,...,...,...,...,...,...,...,...
1014,38123,74914354,22543086,2.384406e+09,26733338325,PNN,M,-172647.42,930715.117000,-1.103363e+06,6.390843
1015,38125,34754804,80849905,4.531743e+09,5789563103,PNN,F,1276506.55,590001.052934,6.865055e+05,0.537800
1016,38132,24770931,48153143,7.842011e+09,80007916156,PNN,M,216333.99,283078.495929,-6.674451e+04,0.308525
1017,38135,5116266,49815187,1.399267e+09,45333816586,PNN,F,413446.98,226059.980521,1.873870e+05,0.453231


In [126]:
dfcom3

1189719.5456392858